<h1 style='color: #000080; font-size: 34px; font-weight: bold;'> Previsão Notas do ENEM 2020
</h1>


# <font color="#228B22" style="font-size: 30px;">Objetivo</font>
<hr style="border: 2px solid #228B22;">

 <p style='font-size: 23px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i><b> Prever o desempenho dos alunos no ENEM 2020 com base nos resultados do ENEM 2019</b></i>     
</p>

---

# <font color='green' style='font-size: 30px;'>Roteiro   </font>
<hr style='border: 2px solid green;'>

<h1 style='color: black; font-size:22px; font-weight: bold;'> 1. Pré-processamento de dados
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'>2. Análise exploratória de dados

</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
3. Modelagem de dados
</h1>


# <font color='green' style='font-size: 30px;'>1. Pré-processamento de dados    </font>
<hr style='border: 2px solid green;'>

In [42]:
# IMPORTANDO BIBLIOTECAS

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [43]:
df = pd.read_csv('MICRODADOS_ENEM_2019_SAMPLE_43278.csv')
df.head(3)

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001004661,2019,1506138,Redenção,15,PA,17,M,1,3,...,A,A,A,A,B,A,C,A,B,B
1,190001004674,2019,1504208,Marabá,15,PA,23,M,1,3,...,A,A,A,B,A,A,B,A,A,B
2,190001004722,2019,1501402,Belém,15,PA,35,F,2,1,...,A,A,A,B,A,B,B,A,A,B


<hr style="border: 1px solid #228B22;">
O aviso de futuro "FutureWarning" ocorre porque a partir de uma versão futura do pandas, o valor padrão de "numeric_only" na função `corr()` será alterado para False. 

Para resolver esse aviso, você pode especificar o parâmetro `numeric_only` explicitamente como True na função `corr()`. Isso garante que apenas as colunas numéricas sejam consideradas no cálculo da correlação. 
<hr style="border: 1px solid #228B22;">


# <h1 style='color: #000080; font-size: 28px; font-weight: bold;'> Remover colunas irrelevantes para o nosso objetivo
</h1>

---

---


## Vamos remover as colunas que tem pouca correlação com nossas colunas de interesse 

---


In [44]:
# Verificando colunas de interesse

df_correlação = df.corr(numeric_only=True)
df_correlação = df_correlação[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']]

display(df_correlação.sort_values(by='NU_NOTA_REDACAO',axis=0, ascending=False).head(15))

colunas_de_interesse = ['NU_INSCRICAO','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
NU_NOTA_REDACAO,0.504903,0.530950,0.537690,0.493200,1.000000
NU_NOTA_COMP3,0.465166,0.491154,0.497266,0.451353,0.935235
NU_NOTA_COMP2,0.411399,0.437303,0.438303,0.398999,0.892802
NU_NOTA_COMP4,0.440447,0.460272,0.464610,0.436736,0.891793
NU_NOTA_COMP1,0.430355,0.468723,0.492001,0.425793,0.851214
NU_NOTA_COMP5,0.457013,0.478832,0.479532,0.443501,0.846894
NU_NOTA_LC,0.641998,0.748844,1.000000,0.587986,0.537690
NU_NOTA_CH,0.669259,1.000000,0.748844,0.610570,0.530950
NU_NOTA_CN,1.000000,0.669259,0.641998,0.650055,0.504903
NU_NOTA_MT,0.650055,0.610570,0.587986,1.000000,0.493200


In [47]:
# FILTRANDO
df = df[colunas_de_interesse]
df.shape

(127380, 11)